In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [23]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED = 6767
tf.random.set_seed(SEED)  

# Get current working directory
cwd = os.getcwd()

We loaded the dataset from Drive.

**Note**: We added a new folder compared to the original folder structure. Specifically, we added a /tt folder that contains the test folder (i.e. /tt/test) in order to properly load the dataset.

In [3]:
# We stored the dataset on drive inside a .zip file
# We load the dataset from drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Unzip dataset from drive folder
!unzip '/content/drive/My Drive/MaskDataset.zip'

Streaming output truncated to the last 5000 lines.
  inflating: MaskDataset/training/11551.jpg  
  inflating: MaskDataset/training/11552.jpg  
  inflating: MaskDataset/training/11553.jpg  
  inflating: MaskDataset/training/11554.jpg  
  inflating: MaskDataset/training/11555.jpg  
  inflating: MaskDataset/training/11556.jpg  
  inflating: MaskDataset/training/11557.jpg  
  inflating: MaskDataset/training/11560.jpg  
  inflating: MaskDataset/training/11561.jpg  
  inflating: MaskDataset/training/11563.jpg  
  inflating: MaskDataset/training/11564.jpg  
  inflating: MaskDataset/training/11565.jpg  
  inflating: MaskDataset/training/11566.jpg  
  inflating: MaskDataset/training/11567.jpg  
  inflating: MaskDataset/training/11568.jpg  
  inflating: MaskDataset/training/11569.jpg  
  inflating: MaskDataset/training/11572.jpg  
  inflating: MaskDataset/training/11574.jpg  
  inflating: MaskDataset/training/11575.jpg  
  inflating: MaskDataset/training/11576.jpg  
  inflating: MaskDataset/trai

### Directory structure

    - MaskDataset/
        - training/
            - img1, img2, …, imgN
        - tt/
            - test/
                - img1, img2, …, imgN

In [24]:
# ImageDataGenerator
# ------------------

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input 

train_data_gen = ImageDataGenerator(rotation_range=3,
                                    # width_shift_range=10,
                                    # height_shift_range=10,
                                    zoom_range=0.1,
                                    # horizontal_flip=True,
                                    # vertical_flip=True,
                                    # fill_mode='constant',
                                    # cval=0,
                                    preprocessing_function=preprocess_input)

# Create validation and test ImageDataGenerator objects
valid_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [25]:
# Create generators to read images from dataset directory
# -------------------------------------------------------
dataset_dir = os.path.join(cwd, 'MaskDataset')

Load the content of the `train_gt.json` file inside a pandas dataframe. We then match the labels from the file to the images in the training dataset.

In [26]:
df = pd.read_json(os.path.join(dataset_dir, 'train_gt.json'), orient='index')
df.reset_index(level=0, inplace=True)
df.columns = ['img', 'label']
df

,img,label
0,14985.jpg,0
1,13358.jpg,0
2,10210.jpg,0
3,18202.jpg,0
4,14962.jpg,0
...,...,...
5609,14383.jpg,2
5610,11787.jpg,2
5611,15770.jpg,2
5612,10934.jpg,2


We match the labels with the images.

We then create a new dataframe and we shuffle it in order to select a subset for training and a subset for validation.

In [27]:
newdf = pd.DataFrame({ 'label': df['label'].map({0: 'A_no_person', 1: 'B_all_people', 2:'C_someone'}), 'img': df['img'] })
newdf = newdf.sample(frac=1, random_state=SEED).reset_index(drop=True)
newdf

,label,img
0,A_no_person,10661.jpg
1,A_no_person,12652.jpg
2,A_no_person,11116.jpg
3,B_all_people,16404.jpg
4,A_no_person,14224.jpg
...,...,...
5609,A_no_person,12578.jpg
5610,C_someone,11160.jpg
5611,B_all_people,13551.jpg
5612,C_someone,12066.jpg


We split the training set between training and validation.

In [28]:
# Create generators to read images from dataset directory
# -------------------------------------------------------
dataset_dir = os.path.join(cwd, 'MaskDataset')

# Batch size
bs = 8

# img shape
img_h = 256
img_w = 256

num_classes=3

# Training
training_dir = os.path.join(dataset_dir, 'training')
train_gen = train_data_gen.flow_from_dataframe(dataframe=newdf.loc[1:3900, :],
                                                directory=training_dir,
                                                x_col="img",
                                                y_col="label",
                                               batch_size=bs, 
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)  # targets are directly converted into one-hot vectors

# Validation
valid_gen = valid_data_gen.flow_from_dataframe(dataframe=newdf.loc[3900:len(newdf), :],
                                                directory=training_dir,
                                                x_col="img",
                                                y_col="label",
                                               batch_size=bs, 
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)

# Test
test_dir = os.path.join(dataset_dir, 'tt')
test_gen = test_data_gen.flow_from_directory(test_dir,
                                             batch_size=bs, 
                                             class_mode='categorical',
                                             shuffle=False,
                                             seed=SEED)

Found 3900 validated image filenames belonging to 3 classes.
Found 1714 validated image filenames belonging to 3 classes.
Found 450 images belonging to 1 classes.


In [29]:
# Create Dataset objects
# ----------------------

# Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Repeat
train_dataset = train_dataset.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Repeat
valid_dataset = valid_dataset.repeat()

# Test
# ----
test_dataset = tf.data.Dataset.from_generator(lambda: test_gen,
                                              output_types=(tf.float32, tf.float32),
                                              output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Repeat
test_dataset = valid_dataset.repeat()

# Transfer Learning - VGG16

In [30]:
# Load VGG16 Model
vgg = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))

In [31]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [32]:
# Create Model
# ------------

# freeze_until = 12

# for layer in vgg.layers[:freeze_until]:
#     layer.trainable = False

vgg.trainable = False
    
model = tf.keras.Sequential()
model.add(vgg)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=512, activation='relu'))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

# Visualize created model as a table
model.summary()

# Visualize initialized weights
model.weights

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 8, 8, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               16777728  
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 1539      
Total params: 31,493,955
Trainable params: 16,779,267
Non-trainable params: 14,714,688
_________________________________________________________________


[<tf.Variable 'block1_conv1/kernel:0' shape=(3, 3, 3, 64) dtype=float32, numpy=
 array([[[[ 4.29470569e-01,  1.17273867e-01,  3.40129584e-02, ...,
           -1.32241577e-01, -5.33475243e-02,  7.57738389e-03],
          [ 5.50379455e-01,  2.08774377e-02,  9.88311544e-02, ...,
           -8.48205537e-02, -5.11389151e-02,  3.74943428e-02],
          [ 4.80015397e-01, -1.72696680e-01,  3.75577137e-02, ...,
           -1.27135560e-01, -5.02991639e-02,  3.48965675e-02]],
 
         [[ 3.73466998e-01,  1.62062630e-01,  1.70863140e-03, ...,
           -1.48207128e-01, -2.35300660e-01, -6.30356818e-02],
          [ 4.40074533e-01,  4.73412387e-02,  5.13819456e-02, ...,
           -9.88498852e-02, -2.96195745e-01, -7.04357103e-02],
          [ 4.08547401e-01, -1.70375049e-01, -4.96297423e-03, ...,
           -1.22360572e-01, -2.76450396e-01, -3.90796512e-02]],
 
         [[-6.13601133e-02,  1.35693997e-01, -1.15694344e-01, ...,
           -1.40158370e-01, -3.77666801e-01, -3.00509870e-01],
    

## Define loss and optimizer

In [33]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

## Fitting the model

In [34]:
model.fit(x=train_dataset,
          epochs=10,
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen),
          )

Epoch 1/10
488/488 [==============================] - 173s 354ms/step - loss: 3.4952 - accuracy: 0.5831 - val_loss: 1.0234 - val_accuracy: 0.6179
Epoch 2/10
488/488 [==============================] - 173s 354ms/step - loss: 0.3402 - accuracy: 0.8669 - val_loss: 1.2734 - val_accuracy: 0.6394
Epoch 3/10
488/488 [==============================] - 172s 352ms/step - loss: 0.1567 - accuracy: 0.9487 - val_loss: 1.7199 - val_accuracy: 0.6167
Epoch 4/10
488/488 [==============================] - 171s 349ms/step - loss: 0.1474 - accuracy: 0.9649 - val_loss: 1.6053 - val_accuracy: 0.6476
Epoch 5/10
488/488 [==============================] - 172s 353ms/step - loss: 0.1469 - accuracy: 0.9662 - val_loss: 2.3059 - val_accuracy: 0.6126
Epoch 6/10
488/488 [==============================] - 172s 353ms/step - loss: 0.1321 - accuracy: 0.9651 - val_loss: 2.4162 - val_accuracy: 0.6593
Epoch 7/10
488/488 [==============================] - 172s 353ms/step - loss: 0.2128 - accuracy: 0.9590 - val_loss: 2.1282 -

## Test model

In [35]:
eval_out = model.evaluate(x=test_dataset,
                          steps=len(test_gen),
                          verbose=0)
eval_out

[4.113815784454346, 0.6315789222717285]

In [36]:
Y_prediction = model.predict(test_gen, len(test_gen))
Y_prediction

array([[3.8452793e-08, 2.8408540e-03, 9.9715912e-01],
       [9.6845543e-01, 1.7815916e-04, 3.1366371e-02],
       [1.5773595e-17, 1.0000000e+00, 5.0230997e-09],
       ...,
       [9.8495564e-18, 9.9899918e-01, 1.0008400e-03],
       [1.2811650e-08, 9.9999404e-01, 5.9295185e-06],
       [9.9999905e-01, 1.9809676e-07, 7.2125027e-07]], dtype=float32)

## Build the .csv file from testset filenames and predictions

In [37]:
array = []
for ch in map(lambda x: x.replace('test/', ''), test_gen.filenames):
  array.append(ch)
array

['10001.jpg',
 '10040.jpg',
 '10074.jpg',
 '10084.jpg',
 '10100.jpg',
 '10120.jpg',
 '10125.jpg',
 '10148.jpg',
 '10213.jpg',
 '10239.jpg',
 '10242.jpg',
 '10259.jpg',
 '10289.jpg',
 '10296.jpg',
 '10323.jpg',
 '10324.jpg',
 '10326.jpg',
 '10328.jpg',
 '10330.jpg',
 '10346.jpg',
 '10349.jpg',
 '10370.jpg',
 '10382.jpg',
 '10396.jpg',
 '10411.jpg',
 '10437.jpg',
 '10459.jpg',
 '10473.jpg',
 '10477.jpg',
 '10494.jpg',
 '10520.jpg',
 '10530.jpg',
 '10571.jpg',
 '10572.jpg',
 '10581.jpg',
 '10620.jpg',
 '10641.jpg',
 '10643.jpg',
 '10662.jpg',
 '10691.jpg',
 '10715.jpg',
 '10727.jpg',
 '10760.jpg',
 '10771.jpg',
 '10782.jpg',
 '10800.jpg',
 '10812.jpg',
 '10845.jpg',
 '10850.jpg',
 '10853.jpg',
 '10863.jpg',
 '10873.jpg',
 '10999.jpg',
 '11012.jpg',
 '11026.jpg',
 '11045.jpg',
 '11115.jpg',
 '11130.jpg',
 '11136.jpg',
 '11137.jpg',
 '11175.jpg',
 '11176.jpg',
 '11202.jpg',
 '11204.jpg',
 '11205.jpg',
 '11210.jpg',
 '11221.jpg',
 '11222.jpg',
 '11233.jpg',
 '11260.jpg',
 '11277.jpg',
 '1128

In [38]:
# Assigning predicted label to each image in the testset
list = []
for i in Y_prediction:
  print(i)
  maxEl = max(i)
  some, = np.where(i == maxEl)
  list.append(some[0])
  print(some)

[3.8452793e-08 2.8408540e-03 9.9715912e-01]
[2]
[9.6845543e-01 1.7815916e-04 3.1366371e-02]
[0]
[1.5773595e-17 1.0000000e+00 5.0230997e-09]
[1]
[6.2801771e-29 9.9999154e-01 8.4274361e-06]
[1]
[0.85223013 0.13322817 0.01454159]
[0]
[5.0154992e-04 8.5288209e-01 1.4661634e-01]
[1]
[2.6107315e-08 2.3245434e-03 9.9767548e-01]
[2]
[0.00000000e+00 9.99999881e-01 1.14769605e-07]
[1]
[1.08419528e-21 1.00000000e+00 1.36098945e-08]
[1]
[1.8301195e-14 9.9998593e-01 1.4106518e-05]
[1]
[9.621622e-01 1.117748e-07 3.783768e-02]
[0]
[2.4900235e-18 9.9993253e-01 6.7468602e-05]
[1]
[3.6162276e-06 6.2670611e-06 9.9999011e-01]
[2]
[1.0000000e+00 4.8992332e-34 2.3166293e-25]
[0]
[1.0011018e-17 9.9999976e-01 2.1059890e-07]
[1]
[0.37049872 0.5457881  0.08371321]
[1]
[8.237769e-09 1.000000e+00 2.140397e-09]
[1]
[0.9935947  0.00137669 0.0050286 ]
[0]
[9.999893e-01 5.181855e-06 5.628816e-06]
[0]
[9.9431252e-01 6.2323579e-08 5.6873704e-03]
[0]
[9.0706952e-17 9.9295866e-01 7.0413388e-03]
[1]
[0.00255955 0.9946906 

In [41]:
series = pd.DataFrame({'Id': array, 'Category': list })
series

,Id,Category
0,10001.jpg,2
1,10040.jpg,0
2,10074.jpg,1
3,10084.jpg,1
4,10100.jpg,0
...,...,...
445,18123.jpg,1
446,18125.jpg,1
447,18153.jpg,1
448,18162.jpg,1


In [42]:
from google.colab import files
import pandas as pd
series.to_csv('example_file.csv', index=False)
files.download('example_file.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>